In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from torchmetrics import Accuracy, Precision, Recall, F1Score

from nltk.tokenize import word_tokenize

from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

## Load the data

0: 20 newsgroups dataset

1: spam dataset

2: twitter dataset

3: amazon reviews dataset

In [2]:
data_set  = 3

### Load the 20 newsgroups dataset

Split the data into train and test


In [3]:
# Load the dataset

if data_set ==0:
    
    categories = ['rec.autos', 'sci.med', 'comp.graphics']
    
    data = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), categories=categories)
    
    # split the data into train and test



### Load the spam dataset

In [33]:
if data_set == 1:
    spam = pd.read_csv('../../assets/spam.csv')
    # do one hot encoding for the target variable
    spam['target'] = pd.get_dummies(spam['target'], drop_first=True, dtype=int)

    # split the data into train and test
    train_set, test_set = 0,0
    
    x_train = train_set['text']
    y_train = train_set['target'].to_numpy()
    
    x_test = test_set['text']
    y_test = test_set['target'].to_numpy()

### Load the twitter dataset

In [5]:
if data_set == 2:
    twitter = pd.read_csv('../../assets/twitter.csv')
    # drop id column
    twitter = twitter.drop('id', axis=1)

    # split the data into train and test
    
    x_train = train_set['tweet']
    y_train = train_set['label'].to_numpy()
    
    x_test = test_set['tweet']
    y_test = test_set['label'].to_numpy()

### Load amazon reviews dataset

In [6]:
if data_set == 3:
    amazon = pd.read_csv('../../assets/amazon_reviews.csv')
    # drop na rows
    amazon = amazon.dropna()
    
    # split the data into train and test

    x_train = train_set['verified_reviews']
    y_train = train_set['feedback'].to_numpy()
    
    x_test = test_set['verified_reviews']
    y_test = test_set['feedback'].to_numpy()

### Vectorize the text data

## Tokenization

Write a function to tokenize the text data

* Remove the punctuation
* Remove stopwords
* Remove non-alphabetic characters
* Convert to lowercase
* Stemming or lemmatization
* Return the tokenized text

In [32]:
def tokenize_1(text):
    # remove the punctuation
    
    # remove stopwords
    
    # remove non-alphabetic characters
    
    # convert to lowercase
    
    # stemming
    
    # lemmatization
    
    return text

## Vectorize the text data

*  max_features = 100
* use CountVectorizer / TfidfVectorizer / ngram_range
    * with the tokenizer function
    * without the tokenizer function
    * with ngam_range
    * with stop_words
    * without stop_words

In [30]:
# Vectorize the text data
max_features = 100


In [ ]:
X_train = vectorizer.fit_transform(x_train)

X_test = vectorizer.transform(x_test)

In [9]:
# Convert feature matrices to PyTorch tensors
X_train_tensor = torch.tensor(X_train.toarray()).float()
X_test_tensor = torch.tensor(X_test.toarray()).float()

# Convert target vectors to PyTorch tensors
y_train_tensor = torch.tensor(y_train)
y_test_tensor = torch.tensor(y_test)

In [11]:
# Reshape the feature matrices
X_train_seq = X_train_tensor.view(X_train_tensor.shape[0], -1, X_train_tensor.shape[1])

In [14]:
# Reshape the feature matrices
X_test_seq = X_test_tensor.view(X_test_tensor.shape[0],-1,X_test_tensor.shape[1])

In [17]:
# Complete the RNN class
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
input_size = 33 # this should be equal to the number of features in X_train_tensor
hidden_size = 32
num_layers = 2
num_classes = 2

In [31]:
# Initialize the model
rnn_model = RNNModel(input_size, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=0.01) 

## Question

Change the optimizer to SGD and ASGD and compare the results

In [21]:

# Train the model for ten epochs and zero the gradients
for epoch in range(10):
    optimizer.zero_grad()
    # outputs = rnn_model(X_train_seq)
    outputs = rnn_model(X_train_seq)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.5903684496879578
Epoch: 2, Loss: 0.3559195399284363
Epoch: 3, Loss: 0.3066563606262207
Epoch: 4, Loss: 0.29748037457466125
Epoch: 5, Loss: 0.29428449273109436
Epoch: 6, Loss: 0.2895662784576416
Epoch: 7, Loss: 0.2821877896785736
Epoch: 8, Loss: 0.27335816621780396
Epoch: 9, Loss: 0.26611509919166565
Epoch: 10, Loss: 0.26439353823661804


## Evaluate the model

Calculate the 
* accuracy
* precision
* recall
* F1 score

In [22]:
# Create an instance of the metrics
accuracy = Accuracy(task="multiclass", num_classes=num_classes)

# create an instance of  precision, recall, and F1 score


## Question

Run the model on the test data

In [34]:
# Generate the predictions
_, predicted = torch.max(outputs, 1)

In [27]:
# Calculate the metrics
accuracy_score = accuracy(predicted, y_test_tensor)

# Calculate the precision, recall, and F1 score


In [28]:
print("RNN Model - Accuracy: {}, Precision: {}, Recall: {}, F1 Score: {}".format(accuracy_score, precision_score, recall_score, f1_score))

RNN Model - Accuracy: 0.8999999761581421, Precision: 0.8999999761581421, Recall: 0.8999999761581421, F1 Score: 0.8999999761581421
